In [1]:
import gym
import random

In [2]:
env = gym.make ('SpaceInvaders-v4', render_mode='rgb_array')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


In [3]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [4]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    
    while not done:

        env.render()
        action = random.choice([0,1,2,3,4,5])
        n_state, reward, done, info, temp = env.step(action)
        score+=reward
    print('Iteration :{} Score :{}'.format(episode,score))
env.close()
    


/home/bleak/.local/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:289: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(
/home/bleak/.local/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Iteration :1 Score :135.0
Iteration :2 Score :105.0
Iteration :3 Score :240.0
Iteration :4 Score :105.0
Iteration :5 Score :410.0


In [5]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam


2023-07-24 23:06:46.146793: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-24 23:06:46.185127: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-24 23:06:46.185526: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-24 23:06:46.678613: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation = 'relu'))
    model.add(Dense(256, activation = 'relu'))
    model.add(Dense(actions, activation = 'linear'))
    
    return model

In [14]:
del model

In [15]:
model = build_model(height, width, channels, actions)

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 3, 51, 39, 32)     6176      
                                                                 
 conv2d_4 (Conv2D)           (None, 3, 24, 18, 64)     32832     
                                                                 
 conv2d_5 (Conv2D)           (None, 3, 22, 16, 64)     36928     
                                                                 
 flatten_1 (Flatten)         (None, 67584)             0         
                                                                 
 dense_3 (Dense)             (None, 512)               34603520  
                                                                 
 dense_4 (Dense)             (None, 256)               131328    
                                                                 
 dense_5 (Dense)             (None, 6)                

In [17]:
from rl.agents import DQNAgent  
from rl.policy import BoltzmannQPolicy  
from rl.memory import SequentialMemory

In [18]:
agent = DQNAgent(
    model=model,
    memory=SequentialMemory(limit=50000, window_length=1),
    policy=BoltzmannQPolicy(),
    nb_actions=actions,
    nb_steps_warmup=10,
    target_model_update=0.01
)

In [19]:
from tensorflow.keras.optimizers.legacy import Adam
agent.compile(Adam(learning_rate=1e-3))

2023-07-24 23:07:16.058443: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled
2023-07-24 23:07:16.078638: W tensorflow/c/c_api.cc:300] Operation '{name:'dense_5_1/bias/Assign' id:632 op device:{requested: '', assigned: ''} def:{{{node dense_5_1/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_5_1/bias, dense_5_1/bias/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


In [20]:
agent.fit(env, nb_steps=10000, visualize=False, verbose=2)

Training for 10000 steps ...


ValueError: Error when checking input: expected conv2d_3_input to have 5 dimensions, but got array with shape (1, 1, 2)